In [1]:
import torch
from torch import nn
import numpy as np
import random
from collections import deque
from itertools import count
import torch.nn.functional as F
from mdp import TradeExecutionEnv, DiscreteTradeSizeWrapper, RelativeTradeSizeWrapper
from tensorboardX import SummaryWriter
from torch.distributions import Categorical


SEED = 42
HORIZON = 5 * 12 * 2
UNITS_TO_SELL = 64
BATCH_SIZE = 32
EPOCHS = 300
EVAL_EPOCHS = 50
EPSILON = 0.1
GAMMA = 0.99

env = TradeExecutionEnv()

trade_sizes = {
  i: i*2 for i in range(33)
}
env = DiscreteTradeSizeWrapper(env, trade_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class CategoricalPolicy(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim=32) -> None:
        super().__init__()
        self.rnn = nn.LSTM(num_states, hidden_dim, 1, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_actions)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = F.relu(self.fc1(x[:, -1, :]))
        x = F.softmax(self.fc2(x), dim=1)
        return x

    def select_action(self, state):
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [2]:
def parse_state(state):
    data = torch.FloatTensor(np.stack([
        state["low"].to_numpy(),
        state["high"].to_numpy(),
        state["close"].to_numpy(),
        state["open"].to_numpy(),
        state["volume"].to_numpy(),
    ])).T
    return torch.concat([
        data,
        torch.repeat_interleave(torch.FloatTensor([[state["units_sold"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["cost_basis"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["steps_left"]]]), 6, 0),
    ], dim=1).unsqueeze(0)

In [3]:
def sample_rollouts(policy, env, num_rollouts, seed=SEED):
    trajs = []
    for e in range(num_rollouts):
        tau = []
        state = env.reset(UNITS_TO_SELL, HORIZON, seed=(seed * num_rollouts) + e)
        state = parse_state(state)
        done = False
        while not done:
            with torch.no_grad():
                action, _ = policy.select_action(state)
            next_state, reward, done, _, _ = env.step(action)
            next_state = parse_state(next_state)
            tau.append((state, action, reward))
            state = next_state
        states, actions, rewards = zip(*tau)
        states = torch.cat(states)
        actions = torch.tensor([actions])
        rewards = torch.tensor([rewards])
        trajs.append((states, actions, rewards))
    return trajs

In [4]:
def R_tau(r, gamma):
    gammas = torch.tensor([gamma**i for i in range(len(r))])
    return torch.sum(gammas * r)

def avg_batch_rewards(trajs, gamma):
    return torch.mean(torch.stack([R_tau(r, gamma) for _, _, r in trajs]).float())

def grad_log_pi(policy, states, actions):
    logits = policy(states)
    log_probs = torch.log(logits)
    log_probs = log_probs.gather(1,actions).T.flatten()
    g = [torch.autograd.grad(log_p, policy.parameters(), retain_graph=True) for log_p in log_probs]
    return g

In [5]:
def grad_log_tau(policy, tau):
  g = grad_log_pi(policy, tau[0], tau[1])
  stacked_g = [torch.stack(g_).view(len(tau[0]), -1) for g_ in zip(*g)]
  return torch.cat(stacked_g, axis=1).sum(axis=0)

def grad_U_tau(policy, tau, gamma):
  return grad_log_tau(policy, tau) * R_tau(tau[2], gamma)


In [6]:
policy = CategoricalPolicy(len(env.observation_space), len(trade_sizes)).to(device)

train_rewards = []
for e in range(EPOCHS):
  trajs = sample_rollouts(policy, env, BATCH_SIZE, seed=e)
  u = torch.stack([grad_U_tau(policy, tau, GAMMA) for tau in trajs]).mean(axis=0)
  #print(f"U = {u}")
  #print(f"U @ U = {u @ u}")
  g = u * torch.sqrt(2 * EPSILON / (u @ u))
  #print(f"G: {g}")
  n = 0
  for i, p in enumerate(policy.parameters()):
    num_elements = p.numel()
    p.data += g[n:n+num_elements].view(p.shape)
    n += num_elements
  policy.zero_grad()
  print(f"iter {e}: {avg_batch_rewards(trajs, GAMMA)}")
  train_rewards.append(avg_batch_rewards(trajs, GAMMA).item())

iter 0: -9.38464069366455
iter 1: -9.714813232421875
iter 2: -9.106985092163086
iter 3: -8.228470802307129
iter 4: -9.381178855895996
iter 5: -7.857438087463379
iter 6: -8.508143424987793
iter 7: -9.100409507751465
iter 8: -8.202279090881348
iter 9: -8.540745735168457
iter 10: -7.622474670410156
iter 11: -7.9443488121032715
iter 12: -5.199665069580078
iter 13: -4.033417701721191
iter 14: -2.0131030082702637
iter 15: -0.48607712984085083
iter 16: -1.6819502115249634
iter 17: -1.4211323261260986
iter 18: -0.4223829209804535
iter 19: -0.5179128646850586
iter 20: -1.362517237663269
iter 21: -1.120136022567749
iter 22: -0.8169094920158386
iter 23: -0.47875285148620605
iter 24: -0.7495009899139404
iter 25: -0.533865213394165
iter 26: -0.5022906064987183
iter 27: -2.0112416744232178
iter 28: -1.0946060419082642
iter 29: -0.4725147485733032
iter 30: -1.1553064584732056
iter 31: -0.387145459651947
iter 32: -0.4235239624977112
iter 33: -2.042233943939209
iter 34: -0.7791088819503784
iter 35: -0.

In [7]:
same_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    same_mdp_eval_rewards.append(episode_rewards)
print(f"Avg same mdp eval reward: {np.mean(same_mdp_eval_rewards)}")

Avg same mdp eval reward: -0.8490901733027204


In [8]:
train_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=(BATCH_SIZE * EPOCHS)+e)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    train_mdp_eval_rewards.append(episode_rewards)
print(f"Avg train mdp eval reward: {np.mean(train_mdp_eval_rewards)}")

Avg train mdp eval reward: -0.4632913543858452


In [9]:
test_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED+e, test=True)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    test_mdp_eval_rewards.append(episode_rewards)
print(f"Avg test mdp eval reward: {np.mean(test_mdp_eval_rewards)}")

Avg test mdp eval reward: -0.45467483205719217


In [11]:
import json

EXP_NAME = "restricted_policy_update_meta_mdp_train.json"
with open(f"./results/{EXP_NAME}", "w+") as f:
    json.dump({
        "train_rewards": train_rewards,
        "same_mdp_eval_rewards": same_mdp_eval_rewards,
        "train_mdp_eval_rewards": train_mdp_eval_rewards,
        "test_mdp_eval_rewards": test_mdp_eval_rewards
    }, f)